In [11]:
# Ignore warnings
import warnings
warnings.filterwarnings("ignore")

In [12]:
%pip install langchain -q
%pip install ibm-watson-machine-learning -q
%pip install wget -q
%pip install sentence-transformers -q

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [13]:
%pip install singlestoredb -q
%pip install sqlalchemy-singlestoredb -q

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [14]:
import os
import getpass
import shutil
import argparse

In [23]:
try:
    wxa_url = os.getenv("WATSONX_URL")
except KeyError:
    wxa_url = getpass.getpass("Please enter your watsonx.ai URL domain (hit enter): ")

In [24]:
try:
    wxa_api_key = os.getenv("WXA_API_KEY")
except KeyError:
    wxa_api_key = getpass.getpass("Please enter your watsonx.ai API key (hit enter): ")

In [25]:
try:
    wxa_project_id = os.getenv("WXA_PROJECT_ID")
except KeyError:
    wxa_project_id = getpass.getpass("Please enter your watsonx.ai Project ID (hit enter): ")

In [26]:
from langchain_community.embeddings import HuggingFaceEmbeddings

def get_embedding_function():
    embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
    return embeddings

In [19]:
from langchain.document_loaders.pdf import PyPDFDirectoryLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain.schema.document import Document
from langchain_community.vectorstores import Chroma

In [20]:
CHROMA_PATH = "chroma"
DATA_PATH = "data"

In [22]:
def load_documents():
    document_loader = PyPDFDirectoryLoader(DATA_PATH)
    return document_loader.load()

In [27]:
def split_documents(documents: list[Document]):
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=800,
        chunk_overlap=80,
        length_function=len,
        is_separator_regex=False,
    )
    return text_splitter.split_documents(documents)

In [28]:
def calculate_chunk_ids(chunks):

    # This will create IDs like "data/monopoly.pdf:6:2"
    # Page Source : Page Number : Chunk Index

    last_page_id = None
    current_chunk_index = 0

    for chunk in chunks:
        source = chunk.metadata.get("source")
        page = chunk.metadata.get("page")
        current_page_id = f"{source}:{page}"

        # If the page ID is the same as the last one, increment the index.
        if current_page_id == last_page_id:
            current_chunk_index += 1
        else:
            current_chunk_index = 0

        # Calculate the chunk ID.
        chunk_id = f"{current_page_id}:{current_chunk_index}"
        last_page_id = current_page_id

        # Add it to the page meta-data.
        chunk.metadata["id"] = chunk_id

    return chunks

In [29]:
def add_to_chroma(chunks: list[Document]):
    db = Chroma(persist_directory=CHROMA_PATH, embedding_function=get_embedding_function())

    chunks_with_ids = calculate_chunk_ids(chunks)

    existing_items = db.get(include=[]) 
    existing_ids = set(existing_items["ids"])
    print(f"Number of existing documents in DB: {len(existing_ids)}")

    new_chunks = []
    for chunk in chunks_with_ids:
        if chunk.metadata["id"] not in existing_ids:
            new_chunks.append(chunk)

    if len(new_chunks):
        print(f"👉 Adding new documents: {len(new_chunks)}")
        new_chunk_ids = [chunk.metadata["id"] for chunk in new_chunks]
        db.add_documents(new_chunks, ids=new_chunk_ids)
        db.persist()
    else:
        print("✅ No new documents to add")

In [30]:
def clear_database():
    if os.path.exists(CHROMA_PATH):
        shutil.rmtree(CHROMA_PATH)


In [61]:
def main():

    # Check if the database should be cleared (using the --clear flag).
    # parser = argparse.ArgumentParser()
    # parser.add_argument("--reset", action="store_true", help="Reset the database.")
    # args = parser.parse_args()
    # if args.reset:
    #     print("✨ Clearing Database")
    #     clear_database()

    # Create (or update) the data store.
    documents = load_documents()
    chunks = split_documents(documents)
    add_to_chroma(chunks)

In [62]:
if __name__ == "__main__":
    main()

Ignoring wrong pointing object 302 0 (offset 0)
Ignoring wrong pointing object 368 0 (offset 0)


Number of existing documents in DB: 152
👉 Adding new documents: 23


In [35]:
from langchain_community.vectorstores import Chroma
from langchain.prompts import ChatPromptTemplate

In [57]:
PROMPT_TEMPLATE="""
Use the following pieces of information to answer the user's question.
If you don't know the answer, just say that you don't know, don't try to make up an answer.

Context: {context}
Question: {question}

Only return the helpful answer below and nothing else.
Helpful answer:
"""

In [37]:
from ibm_watson_machine_learning.foundation_models.utils.enums import ModelTypes
from ibm_watson_machine_learning.foundation_models import Model
from ibm_watson_machine_learning.metanames import GenTextParamsMetaNames as GenParams
from ibm_watson_machine_learning.foundation_models.utils.enums import DecodingMethods

In [38]:
parameters = {
    GenParams.DECODING_METHOD: DecodingMethods.GREEDY,
    GenParams.MIN_NEW_TOKENS: 1,
    GenParams.MAX_NEW_TOKENS: 50
}

In [45]:
credentials = {
    "url": "https://us-south.ml.cloud.ibm.com",
    "apikey": "YCoYQvKR_lOrxtxuKWERRBB0iM_-GcFeLxv7CzUU_wya"
}

In [47]:
# model = ModelTypes.GRANITE_13B_CHAT_V2

model = Model(
    model_id=ModelTypes.GRANITE_13B_CHAT_V2,
    params=parameters,
    credentials={
        "url": credentials.get("url"),
        "apikey": credentials.get("apikey")
    },
    project_id="3aef9d25-4dd9-48a9-8624-2c6c8c483cc3"
)
from ibm_watson_machine_learning.foundation_models.extensions.langchain import WatsonxLLM
granite_llm_ibm = WatsonxLLM(model=model)
# from langchain.llms import WatsonxLLM

# granite_llm_ibm = WatsonxLLM(model_id=model,
#     url=os.getenv("WATSONX_URL"),
#     apikey=os.getenv("WXA_API_KEY"),
#     project_id=os.getenv("WXA_PROJECT_ID"),
#     params=parameters)

In [58]:
def query_rag(query_text: str):
    db = Chroma(persist_directory=CHROMA_PATH, embedding_function=get_embedding_function())
    
    results = db.similarity_search_with_score(query_text, k=5)

    context_text = "\n\n---\n\n".join([doc.page_content for doc, _score in results])
    prompt_template = ChatPromptTemplate.from_template(PROMPT_TEMPLATE)
    prompt = prompt_template.format(context=context_text, question=query_text)
    # print(prompt)
    response_text = granite_llm_ibm.invoke(prompt)

    sources = [doc.metadata.get("id", None) for doc, _score in results]
    formatted_response = f"Response: {response_text}\nSources: {sources}"
    print(formatted_response)
    return response_text

In [60]:

# query_text = getpass.getpass("Please Enter you query here!")
query_text = input("Please Enter your query here: ")
query_rag(query_text)

Response: Gentamicin is a glycopeptide antibiotic that inhibits protein synthesis in bacteria. It is often used to treat severe infections caused by Gram-positive bacteria, such as meningitis and pneumonia.
Sources: ['data/MedChem9_Antibac.pdf:11:0', 'data/Antibioitcs_guide_2013.pdf:27:2', 'data/Antibioitcs_guide_2013.pdf:25:0', 'data/MedChem9_Antibac.pdf:12:0', 'data/MedChem9_Antibac.pdf:47:0']


'Gentamicin is a glycopeptide antibiotic that inhibits protein synthesis in bacteria. It is often used to treat severe infections caused by Gram-positive bacteria, such as meningitis and pneumonia.'